<a href="https://colab.research.google.com/github/vchandraiitk/datascience/blob/main/Vikas_M5_NB_MiniProject_1_Medical_Q%26A_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Certification Program in Computational Data Science
## A programme by IISc and TalentSprint
### Mini-Project: Medical Q&A using GPT2

## Learning Objectives

At the end of the experiment, you will be able to:

* perform data preprocessing, EDA and feature extraction on the Medical Q&A dataset
* load a pre-trained tokenizer
* finetune a GPT-2 language model for medical question-answering

## Dataset Description

The dataset used in this project is the *Medical Question Answering Dataset* ([MedQuAD](https://github.com/abachaa/MedQuAD/tree/master)). It includes medical question-answer pairs along with additional information, such as the question type, the question *focus*, its UMLS(Unified Medical Language System) details like - Concept Unique Identifier(*CUI*) and Semantic *Type* and *Group*.

To know more about this data's collection, and construction method, refer to this [paper](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-019-3119-4).

The data is extracted and is in CSV format with below features:

- **Focus**: the question focus
- **CUI**: concept unique identifier
- **SemanticType**
- **SemanticGroup**
- **Question**
- **Answer**

## Part-A: Grading = 10 Points

## Information

Healthcare professionals often have to refer to medical literature and documents while seeking answers to medical queries. Medical databases or search engines are powerful resources of upto date medical knowledge. However, the existing documentation is large and makes it difficult for professionals to retrieve answers quickly in a clinical setting. The problem with search engines and informative retrieval engines is that these systems return a list of documents rather than answers. Instead, healthcare professionals can use question answering systems to retrieve short sentences or paragraphs in response to medical queries. Such systems have the biggest advantage of generating answers and providing hints in a few seconds.

### Problem Statement

Fine-tune gpt2 model on medical-question-answering-dataset for performing response generation for medical queries.

Please refer to ***M6 Assignment-1 Fine-tune GPT2*** to get familiar with how to load pre-trained gpt2 tokenizer and model.

### Import required packages

In [33]:
!pip -q install -U accelerate
!pip -q install -U transformers
!pip -q install torch

In [34]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

import warnings
warnings.filterwarnings('ignore')

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [36]:
import os
os.chdir('/content/drive/MyDrive/ISC/MiniProject/M5_NB_MiniProject_1_1')

In [38]:
#@title Download the dataset
!wget -q https://cdn.iisc.talentsprint.com/AIandMLOps/MiniProjects/Datasets/MedQuAD.csv
!ls | grep ".csv"

MedQuAD.csv
MedQuAD.csv.1


**Exercise 1: Read the MedQuAD.csv dataset**

**Hint:** pd.read_csv()

In [39]:
df = pd.read_csv("MedQuAD.csv")
df.shape

(16412, 6)

In [40]:
df.head()

,Focus,CUI,SemanticType,SemanticGroup,Question,Answer
0,Adult Acute Lymphoblastic Leukemia,C0751606,T191,Disorders,What is (are) Adult Acute Lymphoblastic Leukem...,Key Points - Adult acute lymphoblastic leukemi...
1,Adult Acute Lymphoblastic Leukemia,C0751606,T191,Disorders,What are the symptoms of Adult Acute Lymphobla...,"Signs and symptoms of adult ALL include fever,..."
2,Adult Acute Lymphoblastic Leukemia,C0751606,T191,Disorders,How to diagnose Adult Acute Lymphoblastic Leuk...,Tests that examine the blood and bone marrow a...
3,Adult Acute Lymphoblastic Leukemia,C0751606,T191,Disorders,What is the outlook for Adult Acute Lymphoblas...,Certain factors affect prognosis (chance of re...
4,Adult Acute Lymphoblastic Leukemia,C0751606,T191,Disorders,Who is at risk for Adult Acute Lymphoblastic L...,Previous chemotherapy and exposure to radiatio...


### Pre-processing and EDA

**Exercise 2: Perform below operations on the dataset [0.5 Mark]**

- Handle missing values
- Remove duplicates from data considering `Question` and `Answer` columns

- **Handle missing values**

In [41]:
# YOUR CODE HERE
df.isnull().sum()

,0
Focus,14
CUI,565
SemanticType,597
SemanticGroup,565
Question,0
Answer,5


In [42]:
# Drop missing values
# YOUR CODE HERE
df.dropna(inplace=True)
df.shape

(15810, 6)

In [43]:
df.isna().sum()

,0
Focus,0
CUI,0
SemanticType,0
SemanticGroup,0
Question,0
Answer,0


- **Remove duplicates from data considering `Question` and `Answer` columns**

In [44]:
# Check duplicates
# YOUR CODE HERE
df[df.duplicated(subset=['Question', 'Answer'])].shape

(48, 6)

In [45]:
# Drop duplicates
# YOUR CODE HERE
df.drop_duplicates(subset=['Question', 'Answer'], inplace=True)

In [46]:
# Check duplicates
# YOUR CODE HERE
df[df.duplicated(subset=['Question', 'Answer'])].shape

(0, 6)

**Exercise 3: Display the category name, and the number of records belonging to top 100 categories of `Focus` column [1 Mark]**

In [47]:
# YOUR CODE HERE
df['Focus'].head()

,Focus
0,Adult Acute Lymphoblastic Leukemia
1,Adult Acute Lymphoblastic Leukemia
2,Adult Acute Lymphoblastic Leukemia
3,Adult Acute Lymphoblastic Leukemia
4,Adult Acute Lymphoblastic Leukemia


In [48]:
# Top 100 Focus categories names
# YOUR CODE HERE
output_list = df['Focus'].value_counts()[:100].index.tolist()

### Create Training and Validation set

**Exercise 4: Create training and validation set [2 Marks]**

- Consider 4 samples per `Focus` category, for each top 100 categories, from the dataset (It will give 400 samples for training)

- Consider 1 sample per `Focus` category (different from training set), for each top 100 categories, from the dataset (It will give 100 samples for validation)

In [49]:
df_train = pd.DataFrame(columns=['Focus', 'Question', 'Answer'])
df_val = pd.DataFrame(columns=['Focus', 'Question', 'Answer'])

In [50]:
# YOUR CODE HERE
df_copy = df[['Focus', 'Question', 'Answer']]
df_train = pd.DataFrame(columns=['Focus', 'Question', 'Answer'])
df_val = pd.DataFrame(columns=['Focus', 'Question', 'Answer'])
for i in output_list:
    #print(df_copy.query('Focus == @i').sample(n=4))
    df_train = pd.concat([df_train, df_copy.query('Focus == @i').sample(n=4)])

In [51]:
#df_train.head(), df_train.shape

In [52]:
for i in output_list:
    df_val = pd.concat([df_val, df_copy.drop(df_train.index).query('Focus == @i').sample(n=1)])

In [53]:
#df_val.head(), df_val.shape

### Pre-process `Question` and `Answer` text

**Exercise 5: Perform below tasks: [1.5 Marks]**

- Combine `Question` and `Answer` for train and validation data as shown below:
    - sequence = *'\<question\>' + question-text + '\<answer\>' + answer-text*

- Join the combined text using '\n' into a single string for training and validation separately

- Save the training and validation strings as separate text files

- **Combine Question and Answer for train and val data**

In [54]:
# YOUR CODE HERE
df_train['sequence'] = '<question>' + df_train['Question'] + '<answer>' + df_train['Answer']
df_val['sequence'] = '<question>' + df_val['Question'] + '<answer>' + df_val['Answer']

In [55]:
df_train.head()

,Focus,Question,Answer,sequence
568,Breast Cancer,Who is at risk for Breast Cancer? ?,Key Points - Avoiding risk factors and increas...,<question>Who is at risk for Breast Cancer? ?<...
14925,Breast Cancer,What is (are) Breast Cancer ?,A mammogram can often detect breast changes in...,<question>What is (are) Breast Cancer ?<answer...
14931,Breast Cancer,What is (are) Breast Cancer ?,There are two types of breast-conserving surge...,<question>What is (are) Breast Cancer ?<answer...
561,Breast Cancer,What are the symptoms of Breast Cancer ?,Signs of breast cancer include a lump or chang...,<question>What are the symptoms of Breast Canc...
15453,Prostate Cancer,What are the treatments for Prostate Cancer ?,Regardless of the type of treatment you receiv...,<question>What are the treatments for Prostate...


- **Join the combined text using '\n' into a single string for training and validation separately**

In [56]:
# YOUR CODE HERE
train_text = '\n'.join(df_train['sequence'])
val_text = '\n'.join(df_val['sequence'])

In [57]:
#train_text

- **Save the training and validation strings as text files**

In [58]:
# YOUR CODE HERE
with open('train.txt', 'w') as f:
    f.write(train_text)

with open('val.txt', 'w') as f:
    f.write(val_text)

**Exercise 6: Load pre-trained GPT2Tokenizer [0.5 Mark]**

- Use checkpoint = "gpt2"

In [59]:
# YOUR CODE HERE
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

**Exercise 7: Tokenize train and validation data and form TextDataset objects [0.5 Mark]**

- Use the loaded pre-trained tokenizer
- Use training and validation data saved in text files

In [60]:
# YOUR CODE HERE
train_dataset = TextDataset(tokenizer=tokenizer, file_path='train.txt', block_size=128)
val_dataset = TextDataset(tokenizer=tokenizer, file_path='val.txt', block_size=128)

**Exercise 8: Create a DataCollator object [0.5 Mark]**

In [61]:
# YOUR CODE HERE
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

**Exercise 9: Load pre-trained GPT2LMHeadModel [0.5 Mark]**

In [62]:
# YOUR CODE HERE
model = GPT2LMHeadModel.from_pretrained('gpt2')

**Exercise 10: Fine-tune GPT2 Model [1 Mark]**

- Specify training arguments and create a TrainingArguments object (Use 30 epochs)

- Train a GPT-2 model using the provided training arguments

- Save the resulting trained model and tokenizer to a specified output directory

In [63]:
# Set up the training arguments
# YOUR CODE HERE
model_output_path = "/content/drive/MyDrive/ISC/MiniProject/M5_NB_MiniProject_1/gpt_model"
#model_output_path = "/content/gpt_model"

training_args = TrainingArguments(
    output_dir = model_output_path,
    overwrite_output_dir = True,
    per_device_train_batch_size = 4, # try with 2
    per_device_eval_batch_size = 4,  #  try with 2
    num_train_epochs = 100,
    save_steps = 1_000,
    save_total_limit = 2,
    logging_dir = './logs',
    )

In [64]:
# Train the model
# YOUR CODE HERE
trainer = Trainer(
    model = model,
    args = training_args,
    data_collator = data_collator,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
)

In [65]:
trainer.train()

Step,Training Loss
500,2.586900
1000,2.028000
1500,1.665200
2000,1.358800
2500,1.090800
3000,0.889100
3500,0.700900
4000,0.563200
4500,0.456800
5000,0.371400


TrainOutput(global_step=27200, training_loss=0.2729244961282786, metrics={'train_runtime': 4913.8941, 'train_samples_per_second': 22.121, 'train_steps_per_second': 5.535, 'total_flos': 7100610969600000.0, 'train_loss': 0.2729244961282786, 'epoch': 100.0})

In [66]:
# Save the model
trainer.save_model(model_output_path)

# Save the tokenizer
tokenizer.save_pretrained(model_output_path)

('/content/drive/MyDrive/ISC/MiniProject/M5_NB_MiniProject_1/gpt_model/tokenizer_config.json',
 '/content/drive/MyDrive/ISC/MiniProject/M5_NB_MiniProject_1/gpt_model/special_tokens_map.json',
 '/content/drive/MyDrive/ISC/MiniProject/M5_NB_MiniProject_1/gpt_model/vocab.json',
 '/content/drive/MyDrive/ISC/MiniProject/M5_NB_MiniProject_1/gpt_model/merges.txt',
 '/content/drive/MyDrive/ISC/MiniProject/M5_NB_MiniProject_1/gpt_model/added_tokens.json')

**Exercise 11: Test Model with user input prompts [1 Mark]**

- Create `generate_response()` function that takes a trained *model*, *tokenizer*, and a *prompt* string as input and generates a response using the GPT-2 model

- Test it with some user input prompts

In [79]:
# YOUR CODE HERE
def generate_response(model, tokenizer, prompt, max_length=100):

    input_ids = tokenizer.encode(prompt, return_tensors="pt")      # 'pt' for returning pytorch tensor

    # Create the attention mask and pad token id
    attention_mask = torch.ones_like(input_ids)
    pad_token_id = tokenizer.eos_token_id

    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        attention_mask=attention_mask,
        pad_token_id=pad_token_id
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)

In [80]:
# Load the fine-tuned model and tokenizer

# YOUR CODE HERE
model_output_path = "/content/drive/MyDrive/ISC/MiniProject/M5_NB_MiniProject_1/gpt_model"
my_model = GPT2LMHeadModel.from_pretrained(model_output_path)
my_tokenizer = GPT2Tokenizer.from_pretrained(model_output_path)

In [81]:
# Response from model

# YOUR CODE HERE
prompt = "How to diagnose Breast Cancer"  # Replace with your desired prompt
response = generate_response(my_model, my_tokenizer, prompt)
print("Generated response:", response)

Generated response: How to diagnose Breast Cancer ?<answer>There are two types of breast-conserving surgery -- lumpectomy and partial mastectomy. - Lumpectomy is the removal of the tumor and a small amount of normal tissue around it. A woman who has a lumpectomy almost always has radiation therapy as well. Most surgeons also take out some of the lymph nodes under the arm. Lumpectomy is the removal of the tumor and a small amount of normal tissue around it. A woman who


In [83]:
# Testing with given prompt 1

# YOUR CODE HERE
prompt = "Who is at risk for Colorectal Cancer?"  # Replace with your desired prompt
response = generate_response(my_model, my_tokenizer, prompt)
print("Generated response:", response)

Generated response: Who is at risk for Colorectal Cancer? ?<answer>Scientists don't know exactly what causes colorectal cancer, but they have been able to identify some risk factors for the disease. A risk factor is always a risk factor. Whether you develop colorectal cancer depends on the type of cancer you have, how serious it is, and what best fits your lifestyle. The following risk factors increase the risk of colorectal cancer: Older age Older age is the


In [85]:
# Testing with given prompt 2

# YOUR CODE HERE
prompt = "Who is at risk for Breast Cancer?"  # Replace with your desired prompt
response = generate_response(my_model, my_tokenizer, prompt)
print("Generated response:", response)

Generated response: Who is at risk for Breast Cancer? ?<answer>Key Points - Avoiding risk factors and increasing protective factors may help prevent cancer. - The following are risk factors for breast cancer: - Older age - A personal history of breast cancer or benign (noncancer) breast disease - Inherited risk of breast cancer - Dense breasts - Exposure of breast tissue to estrogen made in the body - Taking hormone therapy for symptoms of menopause - Radiation therapy to the breast or chest - Obesity - Drinking


**Exercise 12: Compare the performance of a *GPT2 model* with the *GPT2 model fine-tuned* on MedQuAD data [1 Mark]**

- Load another pre-trained GPT2LMHeadModel and do not fine-tune it

- To generate response using the untuned model, pass it as a parameter to `generate_response()` function

- Test both models (fine-tuned and untuned) with below user input prompts:

    - "What precautions to take for a healthy life?"
    - "What to do after being diagnosed with cancer?"
    - "What to do when feeling sick?"

In [72]:
# Load a pre-trained GPT2 model, do not finetune it with MedQuAD data

# YOUR CODE HERE

In [73]:
# Testing with finetuned model: prompt 1

# YOUR CODE HERE

In [74]:
# Testing with untuned model: prompt 1

# YOUR CODE HERE

In [75]:
# Testing with finetuned model: prompt 2

# YOUR CODE HERE

In [76]:
# Testing with untuned model: prompt 2

# YOUR CODE HERE

In [77]:
# Testing with finetuned model: prompt 3

# YOUR CODE HERE

In [78]:
# Testing with untuned model: prompt 3

# YOUR CODE HERE